# Multiple dispatch


## 참고문헌

https://github.com/JuliaComputing/JuliaBoxTutorials/tree/master/introductory-tutorials/intro-to-julia (github : JuliaComputing/JuliaBoxTutorials/introductory-tutorials/intro-to-julia/)

Topics:

1. [친근한 예제로 시작하기](#1)
2. [Input의 type 특정지어주기](#2)
3. [연습문제](#3)

### 함께보기

- http://deepstat.tistory.com/45 (01. Getting started)(in English)
- http://deepstat.tistory.com/46 (01. Getting started(한글))
- http://deepstat.tistory.com/47 (02. Strings)(in English)
- http://deepstat.tistory.com/48 (02. Strings(한글))
- http://deepstat.tistory.com/49 (03. Data structures)(in English)
- http://deepstat.tistory.com/50 (03. Data structures(한글))
- http://deepstat.tistory.com/51 (04. Loops)(in English)
- http://deepstat.tistory.com/52 (04. Loops(한글))
- http://deepstat.tistory.com/53 (05. Conditionals)(in English)
- http://deepstat.tistory.com/54 (05. Conditionals(한글))
- http://deepstat.tistory.com/55 (06. Functions)(in English)
- http://deepstat.tistory.com/56 (06. Functions(한글))
- http://deepstat.tistory.com/57 (07. Packages)(in English)
- http://deepstat.tistory.com/58 (07. Packages(한글))
- http://deepstat.tistory.com/59 (08. Plotting)(in English)
- http://deepstat.tistory.com/60 (08. Plotting(한글))
- http://deepstat.tistory.com/61 (09. Julia is fast)(in English)
- http://deepstat.tistory.com/62 (09. Julia is fast(한글))
- http://deepstat.tistory.com/63 (10. Multiple dispatch)(in English)

여기서 우리는 Julia의 주요 특징 중 하나인 **multiple dispatch**에 대해 살펴볼거다.

Multiple dispatch 는 소프트웨어를 *일반적*이고 *빠르게* 만들어준다!

### 친근한 예제로 시작하기 <a id=1></a>

Multiple dispatch를 이해하기 위해서, 이전에 본 적 있는 예제로 시작할거다.

함수를 선언할 때, 아래와 같이 input의 type에 대한 아무런 정보를 주지 않고 만들 수 있다.

In [1]:
f(x) = x^2

f (generic function with 1 method)

그러면 Julia는아래와 같이 input이 그럴듯한지 아닌지 스스로 판단하게 된다.

In [2]:
f(10)

100

In [3]:
f([1, 2, 3])

MethodError: MethodError: no method matching ^(::Array{Int64,1}, ::Int64)
Closest candidates are:
  ^(!Matched::Float16, ::Integer) at math.jl:782
  ^(!Matched::Missing, ::Integer) at missing.jl:120
  ^(!Matched::Missing, ::Number) at missing.jl:93
  ...

### Input의 type 특정지어주기 <a id=2></a>

그러나 Julia에게 어떤 input의 type을 허용하는지 명시적으로 알려줄 수 있다.

예를 들어, String을 input으로 사용하는 `foo` 함수를 작성해 보자.

In [4]:
foo(x::String, y::String) = println("x랑 y 둘 다 string이다!")

foo (generic function with 1 method)

보이다시피, input 다음에 콜론을 2개 써서 `x`와 `y`의 type을 `String`으로 제한시켰다.

이제 `foo`는 `String`에만 작동하고 다른 input에는 작동하지 않는다.

In [5]:
foo("hello", "hi!")

x랑 y 둘 다 string이다!


In [6]:
foo(3, 4)

MethodError: MethodError: no method matching foo(::Int64, ::Int64)

`foo` 함수가 integer(`Int`)에도 작동하게 하기 위해서, `::Int` 를 사용해서 다음과 같이 `foo`를 정의할 수 있다.

In [7]:
foo(x::Int, y::Int) = println("x랑 y 둘 다 integer다!")

foo (generic function with 2 methods)

In [8]:
foo(3, 4)

x랑 y 둘 다 integer다!


이제 `foo`는 integer에도 작동한다. 그러나, 여전히 `x`와 `y`가 `String`일 때도 작동함을 아래를 통해 볼 수 있다.

In [9]:
foo("hello", "hi!")

x랑 y 둘 다 string이다!


이게 multiple dispatch의 핵심이다. 아래와 같이 선언했을 때,

```julia
foo(x::Int, y::Int) = println("My inputs x and y are both integers!")
```
아래 코드는 위의 코드를 덮어쓰기 하거나 대체하지 않는다.
```julia
foo(y::String, y::String)```

대신에, `foo`라고 불리는 ***generic function***에 ***method***를 추가로 더했을 뿐이다.

***generic function***은 특정 연산에 대한 추상적인 개념이다.

예를 들어, generic function `+` 은 덧셈에 대한 개념이다.

***method***는 *특정 argument type*에 대한 generic function의 구현이다.

예를 들어, `+`에는 실수, 정수, 행렬 등등을 수용하는 여러 method가 있다.

`methods`를 이용해서 `foo`에 얼마나 많은 method가 있는지 볼 수 있다.

In [10]:
methods(foo)

# 2 methods for generic function "foo":
[1] foo(x::Int64, y::Int64) in Main at In[7]:1
[2] foo(x::String, y::String) in Main at In[4]:1

참고 : 더하는 데는 얼마나 많은 방법이 있을까?

In [11]:
methods(+)

# 163 methods for generic function "+":
[1] +(x::Bool, z::Complex{Bool}) in Base at complex.jl:277
[2] +(x::Bool, y::Bool) in Base at bool.jl:104
[3] +(x::Bool) in Base at bool.jl:101
[4] +(x::Bool, y::T) where T<:AbstractFloat in Base at bool.jl:112
[5] +(x::Bool, z::Complex) in Base at complex.jl:284
[6] +(a::Float16, b::Float16) in Base at float.jl:392
[7] +(x::Float32, y::Float32) in Base at float.jl:394
[8] +(x::Float64, y::Float64) in Base at float.jl:395
[9] +(z::Complex{Bool}, x::Bool) in Base at complex.jl:278
[10] +(z::Complex{Bool}, x::Real) in Base at complex.jl:292
[11] +(::Missing, ::Missing) in Base at missing.jl:92
[12] +(::Missing) in Base at missing.jl:79
[13] +(::Missing, ::Number) in Base at missing.jl:93
[14] +(level::Base.CoreLogging.LogLevel, inc::Integer) in Base.CoreLogging at logging.jl:106
[15] +(c::BigInt, x::BigFloat) in Base.MPFR at mpfr.jl:353
[16] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt) in Base.GMP at gmp.jl:443
[17] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt) in Base.GMP at gmp.jl:442
[18] +(a::BigInt, b::BigInt, c::BigInt) in Base.GMP at gmp.jl:441
[19] +(x::BigInt, y::BigInt) in Base.GMP at gmp.jl:412
[20] +(x::BigInt, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.GMP at gmp.jl:449
[21] +(x::BigInt, c::Union{Int16, Int32, Int64, Int8}) in Base.GMP at gmp.jl:455
[22] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat) in Base.MPFR at mpfr.jl:503
[23] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat) in Base.MPFR at mpfr.jl:496
[24] +(a::BigFloat, b::BigFloat, c::BigFloat) in Base.MPFR at mpfr.jl:490
[25] +(x::BigFloat, c::BigInt) in Base.MPFR at mpfr.jl:349
[26] +(x::BigFloat, y::BigFloat) in Base.MPFR at mpfr.jl:318
[27] +(x::BigFloat, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.MPFR at mpfr.jl:325
[28] +(x::BigFloat, c::Union{Int16, Int32, Int64, Int8}) in Base.MPFR at mpfr.jl:333
[29] +(x::BigFloat, c::Union{Float16, Float32, Float64}) in Base.MPFR at mpfr.jl:341
[30] +(x::Dates.CompoundPeriod, y::Dates.CompoundPeriod) in Dates at /home/yt/Julia/julia/usr/share/julia/stdlib/v1.0/Dates/src/periods.jl:349
[31] +(x::Dates.CompoundPeriod, y::Dates.Period) in Dates at /home/yt/Julia/julia/usr/share/julia/stdlib/v1.0/Dates/src/periods.jl:347
[32] +(x::Dates.CompoundPeriod, y::Dates.TimeType) in Dates at /home/yt/Julia/julia/usr/share/julia/stdlib/v1.0/Dates/src/periods.jl:385
[33] +(x::Dates.Date, y::Dates.Day) in Dates at /home/yt/Julia/julia/usr/share/julia/stdlib/v1.0/Dates/src/arithmetic.jl:78
[34] +(x::Dates.Date, y::Dates.Week) in Dates at /home/yt/Julia/julia/usr/share/julia/stdlib/v1.0/Dates/src/arithmetic.jl:76
[35] +(dt::Dates.Date, z::Dates.Month) in Dates at /home/yt/Julia/julia/usr/share/julia/stdlib/v1.0/Dates/src/arithmetic.jl:59
[36] +(dt::Dates.Date, y::Dates.Year) in Dates at /home/yt/Julia/julia/usr/share/julia/stdlib/v1.0/Dates/src/arithmetic.jl:32
[37] +(dt::Dates.Date, t::Dates.Time) in Dates at /home/yt/Julia/julia/usr/share/julia/stdlib/v1.0/Dates/src/arithmetic.jl:20
[38] +(t::Dates.Time, dt::Dates.Date) in Dates at /home/yt/Julia/julia/usr/share/julia/stdlib/v1.0/Dates/src/arithmetic.jl:24
[39] +(x::Dates.Time, y::Dates.TimePeriod) in Dates at /home/yt/Julia/julia/usr/share/julia/stdlib/v1.0/Dates/src/arithmetic.jl:82
[40] +(dt::Dates.DateTime, z::Dates.Month) in Dates at /home/yt/Julia/julia/usr/share/julia/stdlib/v1.0/Dates/src/arithmetic.jl:52
[41] +(dt::Dates.DateTime, y::Dates.Year) in Dates at /home/yt/Julia/julia/usr/share/julia/stdlib/v1.0/Dates/src/arithmetic.jl:28
[42] +(x::Dates.DateTime, y::Dates.Period) in Dates at /home/yt/Julia/julia/usr/share/julia/stdlib/v1.0/Dates/src/arithmetic.jl:80
[43] +(B::BitArray{2}, J::LinearAlgebra.UniformScaling) in LinearAlgebra at /home/yt/Julia/julia/usr/share/julia/stdlib/v1.0/LinearAlgebra/src/uniformscaling.jl:88
[44] +(a::Pkg.Resolve.VersionWeights.VersionWeight, b::Pkg.Resolve.VersionWeights.VersionWeight) in Pkg.Resolve.VersionWeights at /home/yt/Julia/julia/usr/share/juli

그래서 이제 `foo`는 integer와 string에 대해서 사용할 수 있다. 특정 input에 `foo`를 사용하면, Julia는 어떤 타입의 input인지 추론하고, 적절한 method를 빠르게 전달한다. *이것*이 multiple dispatch다. 

Multiple dispatch는 우리 코드를 일반적이고 빠르게 만든다. 우리 코드는 일반적이고 유연해질 수 있는데, 왜냐하면 특정 구현(specific implementations)으로 코드를 쓰는 것이 아니라 덧셈과 곱셈 같은 추상 연산(abstract operations)으로 코드를 쓸 수 있기 때문이다. 동시에, 빨리 실행되는데, 그 이유는 관련된 타입에 있어서 효율적인 method를 불러낼 수 있기 때문이다.

generic function을 불러낼 때, 어떤 method가 전달되는지 보기 위해서는, @which macro를 사용한다.

In [12]:
@which foo(3, 4)

foo(x::Int64, y::Int64) in Main at In[7]:1

더하기에도 @which를 사용해보자.

In [13]:
@which 3.0 + 3.0

+(x::Float64, y::Float64) in Base at float.jl:395

함수 `foo`에 다른 method를 더 적용시킬 수 있다. `Int`나 `Float64` 등등 숫자로 생각되는 모든 object를 포함하는 **추상적인 type**인 `number`를 사용하는 method를 추가하자.

In [14]:
foo(x::Number, y::Number) = println("x랑 y 둘 다 number이다!")

foo (generic function with 3 methods)

This method for `foo` will work on, for example, floating point numbers:

In [15]:
foo(3.0, 4.0)

x랑 y 둘 다 number이다!


또한 이전에 한 적이 있는, any type을 input으로 받는 duck-typed method를 `foo`에 추가할 수 있다.

In [16]:
foo(x, y) = println("나는 any type을 input으로 넣어도 실행된다!")

foo (generic function with 4 methods)

지금까지 `foo`에 적용한 method를 감안할 때,이 method는 non-numbers를 `foo`의 input으로 넣을 때 마다 적용될 것이다.

In [17]:
v = rand(3)
foo(v, v)

나는 any type을 input으로 넣어도 실행된다!


### 연습문제 <a id=3></a>

#### 10.1

함수 `foo`를 확장하자. `Bool` type 하나를 input으로 넣을 때, "foo with one boolean!" 이라는 문구가 출력되는 method를 추가하자.

In [18]:
foo(x::Bool) = println("boolean 한 개로 실행됐다!")

foo (generic function with 5 methods)

In [19]:
methods(foo)

# 5 methods for generic function "foo":
[1] foo(x::Bool) in Main at In[18]:1
[2] foo(x::Int64, y::Int64) in Main at In[7]:1
[3] foo(x::String, y::String) in Main at In[4]:1
[4] foo(x::Number, y::Number) in Main at In[14]:1
[5] foo(x, y) in Main at In[16]:1

#### 10.2

아래 코드를 실행할 때, 위에서 적용한 method가 전달되고 있는지 확인하라.
```julia
foo(true)
```

In [20]:
foo(true)

boolean 한 개로 실행됐다!


In [21]:
@which foo(true)

foo(x::Bool) in Main at In[18]:1

In [22]:
@assert foo(true) == "boolean 한 개로 실행됐다!"

boolean 한 개로 실행됐다!


AssertionError: AssertionError: foo(true) == "boolean 한 개로 실행됐다!"